In [1]:
%run ../config/config.ipynb

@@Configuration START@@
batch_size: 128
d_model: 256
n_head: 2
max_len: 128
ffn_hidden: 64
n_layers: 6
drop_prob: 0.1
epochs: 300
init_lr: 0.001
weight_decay: 0.0005
clip: 1
Using MPS as device
project_dir: /Users/ball/Documents/workspace/transformer-tutorial
rawdata_dir: /Users/ball/Documents/workspace/transformer-tutorial/data
data_dir: /Users/ball/Documents/workspace/transformer-tutorial/preprocessed
@@Configuration END@@


In [2]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd 
import torch

/Users/ball/Documents/workspace/transformer-tutorial/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
train_data_path = data_dir / "train.parquet"
test_data_path = data_dir / "test.parquet"
validation_data_path = data_dir / "validation.parquet"

df_train = pd.read_parquet(path=train_data_path)
df_test = pd.read_parquet(path=test_data_path)
df_validation = pd.read_parquet(path=validation_data_path)

class en2kr_Train_Dataset(Dataset): 
    def __init__(self, max_len): 
        self.data = df_train
        
    def __len__(self): 
        return len(self.data) 

    def __getitem__(self, idx): 
        row = self.data.iloc[[idx]]
        en_sentence = row["english"].item()
        kr_sentence = row["korean"].item()
        
        return kr_sentence, en_sentence
        
class en2kr_Test_Dataset(Dataset): 
    def __init__(self, max_len): 
        self.data = df_test
        
    def __len__(self): 
        return len(self.data) 

    def __getitem__(self, idx): 
        row = self.data.iloc[[idx]]
        en_sentence = row["english"].item()
        kr_sentence = row["korean"].item()
        
        return kr_sentence, en_sentence

class en2kr_Validation_Dataset(Dataset): 
    def __init__(self, max_len): 
        self.data = df_validation
        
    def __len__(self): 
        return len(self.data) 

    def __getitem__(self, idx): 
        row = self.data.iloc[[idx]]
        en_sentence = row["english"].item()
        kr_sentence = row["korean"].item()
        
        return kr_sentence, en_sentence

In [9]:
train_dataset = en2kr_Train_Dataset(max_len=max_len)
test_dataset = en2kr_Test_Dataset(max_len=max_len)

train_dataloader = DataLoader(train_dataset,pin_memory=True, drop_last=True, batch_size=batch_size, shuffle=True, generator=torch.Generator(device=device))
test_dataloader = DataLoader(test_dataset,pin_memory=True, drop_last=True, batch_size=batch_size, generator=torch.Generator(device=device))